**Data Examples**

In [28]:
import os
import json
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
## Only run if data is not already downloaded
    
## Download relevant SEDA data
# !curl --output ./data/seda_codebook_admindist_5.0.xlsx "https://stacks.stanford.edu/file/druid:cs829jn7849/seda_codebook_admindist_5.0.xlsx"
# !curl --output ./data/seda_admindist_long_cs_5.0_updated_20240319.csv   "https://stacks.stanford.edu/file/druid:cs829jn7849/seda_admindist_long_cs_5.0_updated_20240319.csv"
# !curl --output ./data/seda_admindist_long_gcs_5.0_updated_20240319.csv  "https://stacks.stanford.edu/file/druid:cs829jn7849/seda_admindist_long_gcs_5.0_updated_20240319.csv"
# !curl --output ./data/GRF19.zip "https://nces.ed.gov/programs/edge/data/GRF19.zip"
# !unzip ./data/GRF19.zip -d ./data

## Download and extract relevant food desert data
# !curl --output ./data/food_atlas.zip "https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/80591/2019%20Food%20Access%20Research%20Atlas%20Data.zip?v=65647"
# !unzip ./data/food_atlas.zip -d ./data

## CDC dataset
# !curl --output ./data/cdc_health_data.csv "https://data.cdc.gov/api/views/mb5y-ytti/rows.csv?accessType=DOWNLOAD"

## Food atlas dataset
# !curl -L -o ./data/health_food_access_all.csv "https://drive.google.com/uc?export=download&id=1L0HB_qGJrIUnZ_In7iB1jGNuhlX5rNNZ"

# !rm -f ./data/*.zip


**Food Atlas Data Example**

In [5]:
food_atlas = pd.read_csv("./data/health_food_access_all.csv")
food_atlas.dtypes

CensusTract          int64
State               object
County              object
Urban                int64
LATracts_half        int64
LATracts10           int64
PovertyRate        float64
LowIncomeTracts      int64
lahunvhalfshare    float64
lahunv10share      float64
MHLTH_CrudePrev    float64
SLEEP_CrudePrev    float64
dtype: object

In [7]:
cdc_df = pd.read_csv("./data/cdc_health_data.csv")
cdc_df.dtypes

StateAbbr                  object
StateDesc                  object
CountyName                 object
CountyFIPS                  int64
TractFIPS                   int64
TotalPopulation             int64
ACCESS2_CrudePrev         float64
ACCESS2_Crude95CI          object
ARTHRITIS_CrudePrev       float64
ARTHRITIS_Crude95CI        object
BINGE_CrudePrev           float64
BINGE_Crude95CI            object
BPHIGH_CrudePrev          float64
BPHIGH_Crude95CI           object
BPMED_CrudePrev           float64
BPMED_Crude95CI            object
CANCER_CrudePrev          float64
CANCER_Crude95CI           object
CASTHMA_CrudePrev         float64
CASTHMA_Crude95CI          object
CERVICAL_CrudePrev        float64
CERVICAL_Crude95CI         object
CHD_CrudePrev             float64
CHD_Crude95CI              object
CHECKUP_CrudePrev         float64
CHECKUP_Crude95CI          object
CHOLSCREEN_CrudePrev      float64
CHOLSCREEN_Crude95CI       object
COLON_SCREEN_CrudePrev    float64
COLON_SCREEN_C

**SEDA County Data Example**
- gcs: grade cohort scale, accounts for differences in state assessments to  allow cross state comparison analysis
- cs: cohort scale, standardized for a particular point in time and used for local analysis across grade levels.

**Build tract column**

In [2]:
tract_mapping = pd.read_excel("./data/grf19_lea_tract.xlsx")
print(tract_mapping.head())

seda_gcs = pd.read_csv("./data/seda_admindist_long_gcs_5.0_updated_20240319.csv")
print(seda_gcs.head())

    LEAID                   NAME_LEA19       TRACT  COUNT   LANDAREA  \
0  100001  Fort Rucker School District  1031010300      2  23.428498   
1  100001  Fort Rucker School District  1045020000      2  66.513225   
2  100003  Maxwell AFB School District  1101000900      3   3.356590   
3  100003  Maxwell AFB School District  1101001000      3   0.001526   
4  100003  Maxwell AFB School District  1101006000      3   0.003588   

   WATERAREA  
0   0.000000  
1   1.081745  
2   0.143795  
3   0.000000  
4   0.000000  
   fips stateabb  sedaadmin           sedaadminname subject  grade  year  \
0     1       AL     100002  Alabama Youth Services     mth      8  2009   
1     1       AL     100002  Alabama Youth Services     rla      8  2009   
2     1       AL     100002  Alabama Youth Services     mth      8  2011   
3     1       AL     100002  Alabama Youth Services     rla      8  2011   
4     1       AL     100002  Alabama Youth Services     mth      8  2012   

   gcs_mn_all  gcs_m

In [ ]:
tract_id_mappings = {
    k: v for k, v in zip(tract_mapping["LEAID"], tract_mapping["NAME_LEA19"])
}

seda_id_mappings = {
    k: v for k, v in zip(seda_gcs["sedaadmin"], seda_gcs["sedaadminname"])
}

id_to_tract_mapping = {}
for i, row in tract_mapping.iterrows():
    id_to_tract_mapping[row["LEAID"]] = id_to_tract_mapping.get(row["LEAID"], []) + [row["TRACT"]]

matched = []
for k, v in tract_id_mappings.items():
    if k in seda_id_mappings:
        matched.append(k)
print(len(matched))

In [ ]:
seda_gcs_2019 = seda_gcs[seda_gcs["year"] == 2019]

new_rows = []
for i, row in tqdm(seda_gcs_2019.iterrows(), total=len(seda_gcs_2019)):
    tracts = id_to_tract_mapping.get(row["sedaadmin"], [])
    for tract in tracts:
        new_rows.append(row.to_dict())
        new_rows[-1]["tract"] = tract

seda_gcs_2019 = pd.DataFrame(new_rows)

seda_gcs_2019.to_csv("./data/FINAL_seda_gcs_2019.csv", index=False)
len(seda_gcs_2019)

In [ ]:
seda_gcs_2019.to_csv("./data/FINAL_seda_gcs_2019.csv", index=False)
len(seda_gcs_2019)

In [ ]:
# Plot histogram for a categorical column
state_counts = seda_gcs['year'].value_counts()
state_counts = state_counts.sort_index()
plt.figure(figsize=(10, 5))
state_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Year')
plt.ylabel('Count')
plt.title('Histogram of Year counts')
plt.xticks(rotation=45) 
plt.show()

In [ ]:
## Assess rought distributional statistics for a continuous column
# agg_min, agg_max, agg_mean, agg_stedv = [], [], [], []
# for col in seda_gcs.columns:
#     if col.startswith("tot_asmt") and len(col.split("_")) == 3:
#         values = seda_gcs[seda_gcs[col].notna()][col]
#         agg_min.append(values.min())
#         agg_max.append(values.max())
#         agg_mean.append(values.mean())
#         agg_stedv.append(values.std())
# agg_min.sort()
# agg_max.sort()
# agg_mean.sort()
# agg_stedv.sort()
# print(f"{agg_min},\n{agg_max},\n{agg_mean},\n{agg_stedv}")
# print(statistics.mean(agg_min), statistics.mean(agg_max), statistics.mean(agg_mean), statistics.mean(agg_stedv))


**County Coverage**

In [ ]:
with open("state_abbv_mapping.json") as f:
    state_abbv_mapping = json.load(f)

In [ ]:
food_atlas_counties = set()
for state, county in zip(food_atlas["State"], food_atlas["County"]):
    food_atlas_counties.add(f"{state_abbv_mapping[state]}_{' '.join(county.split(' ')[:-1])}".lower())
len(food_atlas_counties)

In [ ]:
seda_counties = set()
for state, county in zip(seda_gcs["stateabb"], seda_gcs["sedacountyname"]):
    seda_counties.add(f"{state}_{' '.join(county.split(' ')[:-1])}".lower())
len(seda_counties)

**Missing counties**

In [ ]:
missing = []
for pair in seda_counties:
    if pair not in food_atlas_counties:
        state, county = pair.split("_")
        missing.append(f"{county}, {state}")
print(f"In SEDA but not in Food Atlas:")
print(missing)

missing = []
for pair in food_atlas_counties:
    if pair not in seda_counties:
        state, county = pair.split("_")
        missing.append(f"{county}, {state}")
print(f"In Food Atlas but not in SEDA:")
print(missing)


**Missing Data in SEDA dataset**

In [ ]:
race_tags = {
    "Asian": "asn",
    "Black": "blk",
    "Hispanic": "hsp",
    "Native American": "nam",
    # "White-Asian Gap": "wag",
    # "White-Black Gap": "wbg",
    # "White-Hispanic Gap": "whg",
    # "White": "wht",
    # "White-Native American Gap": "whg",
}
race_tags_inv = {v: k for k, v in race_tags.items()}

gender_tags = {
    "Female": "fem",
    "Male": "mal",
    # "Male-Female Gap": "mfg"
}
gender_tags_inv = {v: k for k, v in gender_tags.items()}

additional_tags = {
    "Early Childhood Development": "ecd",
    "Non-Early Childhood Development": "nec",
    "Non-ECD ECD Gap": "neg"
}
additional_tags_inv = {v: k for k, v in additional_tags.items()}

# Add g before cs (i.e. "gcs") for the grade comparable scale table
columns = {
    "County Mean": "cs_mn",
    "County STDEV": "cs_mn_se",
    "Total Number of Students": "tot_asmt",
}
columns_inv = {v: k for k, v in columns.items()}

In [ ]:

for state, group in seda_gcs.groupby("stateabb"):
    max_missing = 0
    max_missing_col = ""
    for col, tot in zip(group.isna().sum().keys(), group.isna().sum().values):
        if tot > max_missing:
            if any([x in col for x in ["asn", "blk", "hsp", "nam", "fem", "mal"]]):
                max_missing = tot
                max_missing_col = col
    print(f"{state}'s most sparse column is {max_missing_col} with {max_missing / len(group)} percent missing")

In [ ]:
## Investigate missing values

# State
for state, group in seda_gcs.groupby("stateabb"):
    print(f"Percent of rows with missing values for {state}: {max(group.isna().sum()) / len(group)}")


# Subject
print()
for state, group in seda_gcs.groupby("subject"):
    print(f"Percent of rows with missing values for {state}: {max(group.isna().sum()) / len(group)}")

# Grade
print()
for state, group in seda_gcs.groupby("grade"):
    print(f"Percent of rows with missing values for {state}: {max(group.isna().sum()) / len(group)}")

# Year
print()
for state, group in seda_gcs.groupby("year"):
    print(f"Percent of rows with missing values for {state}: {max(group.isna().sum()) / len(group)}")

# Per Column

**Data Example**

In [36]:
if not os.path.exists("./data/data.db"):
    os.system("python build_db.py")

conn = sqlite3.connect("./data/data.db")
c = conn.cursor()


COLS_EXCLUDE = set([
    "StateAbbr",
    "StateDesc",
    "CountyName",
    "CountyFIPS",
    "TractFIPS",
    "fips",
    "stateabb",
    "sedaadmin",
    "sedaadminname",
    "TRACT",
    "COUNT"
])

c.execute("""
WITH seda_tracts AS (
    SELECT *
    FROM seda s
    JOIN nces n
    ON s.sedaadmin = n.LEAID
    WHERE s.year = 2019
)

, food_atlas_tracts AS (
    SELECT *
    FROM food f
    JOIN seda_tracts st
    ON st.TRACT = f.CensusTract
)

, cdc_tracts AS (
    SELECT *
    FROM cdc c
    JOIN food_atlas_tracts ft
    ON ft.TRACT = c.TractFIPS
)

SELECT *
FROM cdc_tracts
LIMIT 100;
""")

rows = c.fetchall()
print(len(rows))
columns = [col[0] for col in c.description]
df = pd.DataFrame(rows, columns=columns)
df = df[[col for col in df.columns if col not in COLS_EXCLUDE]]
df.to_csv("example_joined_table.csv", index=False)
df



100


,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,BINGE_CrudePrev,BINGE_Crude95CI,BPHIGH_CrudePrev,BPHIGH_Crude95CI,BPMED_CrudePrev,BPMED_Crude95CI,CANCER_CrudePrev,CANCER_Crude95CI,CASTHMA_CrudePrev,CASTHMA_Crude95CI,CERVICAL_CrudePrev,CERVICAL_Crude95CI,CHD_CrudePrev,CHD_Crude95CI,CHECKUP_CrudePrev,CHECKUP_Crude95CI,CHOLSCREEN_CrudePrev,CHOLSCREEN_Crude95CI,COLON_SCREEN_CrudePrev,COLON_SCREEN_Crude95CI,COPD_CrudePrev,COPD_Crude95CI,COREM_CrudePrev,COREM_Crude95CI,COREW_CrudePrev,COREW_Crude95CI,CSMOKING_CrudePrev,CSMOKING_Crude95CI,DENTAL_CrudePrev,DENTAL_Crude95CI,DEPRESSION_CrudePrev,DEPRESSION_Crude95CI,DIABETES_CrudePrev,DIABETES_Crude95CI,GHLTH_CrudePrev,GHLTH_Crude95CI,HIGHCHOL_CrudePrev,HIGHCHOL_Crude95CI,KIDNEY_CrudePrev,KIDNEY_Crude95CI,LPA_CrudePrev,LPA_Crude95CI,MAMMOUSE_CrudePrev,MAMMOUSE_Crude95CI,MHLTH_CrudePrev,MHLTH_Crude95CI,OBESITY_CrudePrev,OBESITY_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,CensusTract,State,County,Urban,LATracts_half,LATracts10,PovertyRate,LowIncomeTracts,lahunvhalfshare,lahunv10share,MHLTH_CrudePrev:1,SLEEP_CrudePrev:1,subject,grade,year,gcs_mn_all,gcs_mn_se_all,tot_asmt_all,gcs_mn_asn,gcs_mn_se_asn,tot_asmt_asn,gcs_mn_blk,gcs_mn_se_blk,tot_asmt_blk,gcs_mn_ecd,gcs_mn_se_ecd,tot_asmt_ecd,gcs_mn_fem,gcs_mn_se_fem,tot_asmt_fem,gcs_mn_hsp,gcs_mn_se_hsp,tot_asmt_hsp,gcs_mn_mal,gcs_mn_se_mal,tot_asmt_mal,gcs_mn_mfg,gcs_mn_se_mfg,tot_asmt_mfg,gcs_mn_nam,gcs_mn_se_nam,tot_asmt_nam,gcs_mn_nec,gcs_mn_se_nec,tot_asmt_nec,gcs_mn_neg,gcs_mn_se_neg,tot_asmt_neg,gcs_mn_wag,gcs_mn_se_wag,tot_asmt_wag,gcs_mn_wbg,gcs_mn_se_wbg,tot_asmt_wbg,gcs_mn_whg,gcs_mn_se_whg,tot_asmt_whg,gcs_mn_wht,gcs_mn_se_wht,tot_asmt_wht,gcs_mn_wng,gcs_mn_se_wng,tot_asmt_wng,last_bie,LEAID,NAME_LEA19,LANDAREA,WATERAREA
0,3724,17.7,"(16.1, 19.2)",34.8,"(33.9, 35.7)",12.9,"(12.6, 13.3)",42.6,"(41.6, 43.4)",78.2,"(77.6, 78.8)",7.6,"( 7.3, 7.8)",10.1,"( 9.8, 10.5)",83.8,"(82.3, 85.2)",8.1,"( 7.6, 8.6)",76.1,"(75.6, 76.7)",86.8,"(86.6, 87.0)",65.9,"(63.0, 68.8)",10.4,"( 9.4, 11.4)",35.4,"(30.7, 40.5)",27.3,"(23.4, 31.5)",22.8,"(20.9, 24.6)",60.6,"(58.3, 63.1)",28.4,"(27.5, 29.3)",13.4,"(12.8, 14.1)",25.2,"(23.2, 27.2)",39.1,"(38.3, 39.8)",3.4,"( 3.2, 3.5)",36.0,"(34.0, 38.0)",76.8,"(74.3, 79.4)",18.6,"(17.6, 19.5)",37.9,"(36.9, 38.9)",17.5,"(16.5, 18.7)",36.9,"(36.1, 38.7)",4.2,"( 3.9, 4.5)",18.1,"(13.6, 23.2)",POINT (-86.27432282 34.31407809),1095030701,Alabama,Marshall County,0,0,0,26.0,1,5.26,None,18.6,36.9,mth,3,2019,2.084955,0.173620,448,None,None,None,None,None,None,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,None,None,None,2.566329,0.202350,248.0,1.077884,0.296578,448.0,None,None,None,None,None,None,0.817592,0.312159,418.0,2.579603,0.241355,166.0,None,None,None,0,100005,Albertville City School District,2.116012,0.000000
1,4024,17.9,"(15.5, 20.4)",33.7,"(32.6, 34.6)",14.1,"(13.6, 14.7)",41.4,"(40.3, 42.5)",76.7,"(75.9, 77.5)",7.3,"( 7.1, 7.6)",10.3,"( 9.8, 10.8)",83.6,"(81.7, 85.2)",7.7,"( 7.2, 8.3)",75.1,"(74.2, 75.8)",85.6,"(84.6, 86.6)",64.8,"(61.5, 67.8)",10.3,"( 9.2, 11.4)",34.1,"(29.1, 39.5)",27.3,"(23.2, 31.7)",24.0,"(21.5, 26.5)",58.9,"(55.5, 61.8)",29.3,"(28.1, 30.4)",12.7,"(12.0, 13.4)",25.3,"(22.7, 27.9)",38.7,"(37.9, 39.6)",3.2,"( 3.0, 3.3)",35.3,"(32.8, 37.8)",75.9,"(72.7, 78.8)",18.9,"(17.7, 20.2)",37.2,"(36.0, 38.3)",17.3,"(16.0, 18.6)",37.3,"(36.0, 38.3)",4.1,"( 3.7, 4.4)",19.9,"(14.6, 25.6)",POINT (-86.2583509 34.35104512),1095030702,Alabama,Marshall County,0,0,0,15.2,0,6.77,None,18.9,37.3,mth,3,2019,2.084955,0.173620,448,None,None,None,None,None,None,1.499328,0.216733,200.0,1.917122,0.203595,225.0,1.737057,0.197765,252.0,2.228891,0.219325,223.0,0.338979,0.299259,448.0,None,None,None,2.566329,0.202350,248.0,1.077884,0.296578,448.0,None,None,None,None,None,None,0.817592,0.312159,4